In [1]:
from qutip_qip.device import Processor,Model
from qiskit import QuantumCircuit
from qutip_qip.qiskit import QiskitPulseSimulator, QiskitCircuitSimulator
from qiskit.compiler import transpile
import numpy as np
from scipy.optimize import minimize
import matplotlib
from qutip import *
from qiskit import *

In [2]:
class RXGate:
    def __init__(self, num_qubits, omega, t0):
        self.num_qubits = num_qubits
        self.omega = omega
        self.t0 = t0
        self.sigma = np.sqrt(2*np.pi)/self.omega

    def set_rotation(self, rotation):
        if rotation < 0:
            rotation = (2*np.pi) - rotation

        self.sigma = float(rotation)/(self.omega*np.sqrt(2*np.pi))

    def Hamiltonain(self, t, target_qubit):
        omega_t = self.omega * np.exp(-(np.square(t-self.t0))/(2*np.square(self.sigma)))
        H = (omega_t/2)*(basis(4,0)*basis(4,1).dag() + basis(4,1)*basis(4,0).dag())
        H_list = [qeye(4)]*self.num_qubits
        H_list[target_qubit] = H
        return tensor(H_list)
          
class RZGate:
    def __init__(self, num_qubits, T, delta):
        self.num_qubits = num_qubits
        self.T =  T
        self.delta = delta
        self.rotation = 0

    def set_rotation(self, rotation):
        if rotation < 0:
            self.rotation = (2*np.pi) - rotation
        else:
            self.rotation = rotation
    
    def Hamiltonain(self, t, target_qubit):
        delta_t = 0
        Tz = self.T * self.rotation
        if t < Tz and t >= 0:
            delta_t = self.delta
        H = delta_t * basis(4,1)*basis(4,1).dag()
        H_list = [qeye(4)]*self.num_qubits
        H_list[target_qubit] = H
        return tensor(H_list)

class CZGate:
    def __init__(self, num_qubits, T, tau, omega, delta, blockade_strength):
        self.num_qubits = num_qubits
        self.T = T
        self.tau = tau
        self.omega = omega
        self.delta = delta
        self.blockade_strength = blockade_strength

    @staticmethod
    def Omega(t, T, tau, omega):
        t0 = 0
        if t < T/2:
            t0 = T/4
        elif t < T:
            t0 = 3*T/4

        a = np.exp(-np.power((T/(4*tau)), 4))
        return omega*((np.exp(-(np.power(((t-t0)/tau),4)))-a)/(1-a))        

    @staticmethod
    def Delta(t, T, delta_r):
        if t < T/2:
            return -delta_r * np.cos((2*np.pi/T)*t)
        else:
            return delta_r * np.cos((2*np.pi/T)*(t-T/2))      
        
    def Hamiltonain(self, t, control_qubit, target_qubit): 
        omega_t = self.Omega(t, self.T, self.tau, self.omega)
        delta_t = self.Delta(t, self.T, self.delta)
        Hc = [qeye(4)]*self.num_qubits
        Ht = [qeye(4)]*self.num_qubits
        rr = [basis(4,0)]*self.num_qubits
        
        Hc[control_qubit] = (omega_t/2)*(basis(4,1)*basis(4,2).dag() + basis(4,2)*basis(4,1).dag())
        Hc[control_qubit] += delta_t * basis(4, 2) * basis(4, 2).dag()

        Ht[target_qubit] = (omega_t/2)*(basis(4,1)*basis(4,2).dag() + basis(4,2)*basis(4,1).dag())
        Ht[target_qubit] += delta_t * basis(4, 2) * basis(4, 2).dag()
        
        rr[target_qubit] = basis(4,2)
        rr[control_qubit] = basis(4,2)
        Hb = self.blockade_strength * tensor(rr) * tensor(rr).dag()
        return (tensor(Hc) + tensor(Ht) + Hb)
    
class MeasureGate:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits

    def measure(self, state, target_qubit):
        proj_list = [basis(4,0)*basis(4,0).dag(), basis(4,1)*basis(4,1).dag()]
        prob_list = []
        rho_list = []
        rho = state.ptrace(target_qubit)
        for proj  in proj_list:
            prob_list.append((rho * proj).tr().real)
        for i in range(num_qubits):
            rho_list.append(rho)
            norm_prob_list = [float(i)/sum(prob_list) for i in prob_list]
            outcome = np.random.choice([0, 1], p = norm_prob_list)
            rho_list[target_qubit] = (proj_list[outcome] * rho_list[target_qubit] * proj_list[outcome]).unit()
        return tensor(rho_list)

In [4]:
class NeutralAtomProcessor(Processor):
    def __init__ (self ,num_qubits=2, decay_rate=0):
        self.model = Model(2)
        self.num_qubits = num_qubits
        self.decay_rate = decay_rate
        
        self.native_gate_set = ["rx", "rz", "cz", "measure"]

        self.gates = []
        self.target_qubits = []
        self.control_qubits = []
        self.args = []

        self.rx_gate = RXGate(num_qubits, omega = 1, t0 = 5)
        self.rz_gate = RZGate(num_qubits, T=np.pi, delta=1)
        self.cz_gate = CZGate(num_qubits, T=0.54, tau=0.54*0.175, omega=17*2*np.pi, delta=23*2*np.pi, blockade_strength=100*2*np.pi)
        self.measure_gate = MeasureGate(num_qubits)
        self.branching_ratio = [1.0/16, 1.0/16, 0, 7.0/8]
        self.c_ops = self.collapse_operator()
        return
    
    def generate_init_processor_state(self):
        qubits = [basis(4,0)]*self.num_qubits
        return tensor(qubits)
        
    def load_circuit(self ,qutip_circuit):
        self.gates = []
        self.target_qubits = []
        self.control_qubits = []
        self.args = []
        i = 1
        for gate in qutip_circuit.gates:
            self.gates.append(gate.name)
            self.target_qubits.append(gate.targets[0])
            if gate.name != "measure":
                self.control_qubits.append(None if gate.controls is None else gate.controls[0])
                self.args.append(gate.arg_value)
            else:
                self.control_qubits.append(None)
                self.args.append(None)
            i = i + 1
        
        print("Circuit has ", len(self.gates), " gates after transpiling")
        return

    def run_state(self, init_state):
        if len(self.gates) == 0:
            result = Result(e_ops=[], options={'store_states':None, 'store_final_state':True})
            result.states.append(init_state)
            result.times.append(0)
            result._final_state_ = init_state
            return result
        
        state = init_state

        for i in range(len(self.gates)):
            gate = self.gates[i]
            target_qubit = int(self.target_qubits[i])
            if gate.lower() == "rx":
                self.rx_gate.set_rotation(float(self.args[i]))
                times = np.linspace(0, 50*self.rx_gate.sigma,  50)
                args = {'self':self.rx_gate, "target_qubit": target_qubit}
                result = mesolve([self.rx_gate.Hamiltonain], state, times, [self.c_ops], [], args = args)
                state = result.states[-1]
            elif gate.lower() == "rz":
                self.rz_gate.set_rotation(float(self.args[i]))
                times = np.linspace(0, 2*np.pi, 50)
                args = {'self':self.rz_gate, "target_qubit": target_qubit}
                result = mesolve([self.rz_gate.Hamiltonain], state, times, [self.c_ops], [], args = args)
                state = result.states[-1]
            elif gate.lower() == "cz":
                times = np.linspace(0, self.cz_gate.T, 50)
                control_qubit = int(self.control_qubits[i])
                args = {'self':self.cz_gate.Hamiltonain, 'control_qubit': control_qubit, 'target_qubit': target_qubit}
                result = mesolve([self.cz_gate.Hamiltonain], state, times, [self.c_ops], [], args = args)
                state = result.states[-1]
            elif gate.lower() == "measure":
                state = self.measure_gate.measure(state, target_qubit)
                result = Result(e_ops=[], options={})
                result.times.append(0)
                result.states.append(state)
                result._final_state = state
        return result

    def get_final_circuit_state(self ,final_state):
        print("Circuit complete")
        final_state_list = []
        for i in range(self.num_qubits):
            state = final_state.ptrace(i)
            qubit_state = []
            for j in range(2):
                qubit_state.append(np.sqrt(state[j,j]))
            final_state_list.append(Qobj(qubit_state).unit())
        return tensor(final_state_list)


    def collapse_operator(self):
        def collapse(decay_rate, branching_ratio):
            L = 0
            r = 2
            for j in [0, 1, 3]:
                L += np.sqrt(branching_ratio[j]*decay_rate)*basis(4,j)*basis(4,2).dag()
            return L
        
        c_ops = []
        for i in range(self.num_qubits):
            tensor_list = [qeye(4)] * self.num_qubits
            tensor_list[i] = collapse(self.decay_rate, self.branching_ratio)
            c_ops.append(tensor(tensor_list))
        
        return sum(c_ops)
    
    def apply_parameters(self, parameters, num_inputs, num_weights):
        for i in range(len(self.gates)):
            arg = self.args[i]
            print(arg)
            if "input" in arg:
                print (arg)
            elif "weight" in arg:
                print(arg)
            else:
                continue